Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [239]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
import math as math
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
train_subset = 10000
l2_penalty = 0.01

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])

  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
 
  logits = tf.matmul(tf_train_dataset,weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = loss + l2_penalty * tf.nn.l2_loss(weights)
  # TODO add L2 regularization to the unregu

  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [6]:
num_steps = 901

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 53.994820
Training accuracy: 4.3%
Validation accuracy: 5.9%
Loss at step 100: 11.697384
Training accuracy: 74.0%
Validation accuracy: 73.5%
Loss at step 200: 4.429626
Training accuracy: 78.8%
Validation accuracy: 77.7%
Loss at step 300: 1.961160
Training accuracy: 81.8%
Validation accuracy: 80.6%
Loss at step 400: 1.125648
Training accuracy: 83.3%
Validation accuracy: 81.8%
Loss at step 500: 0.839356
Training accuracy: 83.8%
Validation accuracy: 82.2%
Loss at step 600: 0.739983
Training accuracy: 83.9%
Validation accuracy: 82.4%
Loss at step 700: 0.705147
Training accuracy: 84.0%
Validation accuracy: 82.5%
Loss at step 800: 0.692838
Training accuracy: 84.0%
Validation accuracy: 82.5%
Loss at step 900: 0.688456
Training accuracy: 84.0%
Validation accuracy: 82.5%
Test accuracy: 88.3%


In [228]:
batch_size = 10
num_nodes = 1024

def runHiddenNetwork(batch_size=128, num_steps = 3001, use_l2 = True, use_dropout = False, prob_val = 1.0) :
    graphRelu = tf.Graph()
    with graphRelu.as_default():
        keep_prob = tf.placeholder(tf.float32)
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)
        # Variables.
        weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
        biases_1 = tf.Variable(tf.zeros([num_nodes]))
        weights_2 = tf.Variable( tf.truncated_normal([num_nodes, num_labels]))
        biases_2 = tf.Variable(tf.zeros([num_labels]))
        logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
      
        relu_layer = tf.nn.relu(logits_1)  
        if(use_dropout) :
            relu_layer = tf.nn.dropout(relu_layer,keep_prob)
        logits_2 = tf.matmul(relu_layer, weights_2) + biases_2 # out layer --------------------------------
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits_2,labels=tf_train_labels))
        if(use_l2):
            loss = loss + l2_penalty * tf.nn.l2_loss(weights_1)  + l2_penalty * tf.nn.l2_loss(weights_2)
        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
        train_prediction = tf.nn.softmax(logits_2)
        
        logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
        relu_layer = tf.nn.relu(logits_1)
        logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
        test_prediction =  tf.nn.softmax(logits_2)       
        
        with tf.Session(graph=graphRelu) as session:
            tf.initialize_all_variables().run()
            print("Initialized")
            for step in range(num_steps):
                # Pick an offset within the training data, which has been randomized.
                # Note: we could use better randomization across epochs.
                offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
                # Generate a minibatch.
                batch_data = train_dataset[offset:(offset + batch_size), :]
                batch_labels = train_labels[offset:(offset + batch_size), :]
                # Prepare a dictionary telling the session where to feed the minibatch.
                # The key of the dictionary is the placeholder node of the graph to be fed,
                # and the value is the numpy array to feed to it.
                feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:prob_val}
                _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
                if (step % 500 == 0):
                    print("Minibatch loss at step {}: {}".format(step, l))
                    print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))
            #test_prediction.eval()

In [229]:
runHiddenNetwork(batch_size=128, use_l2 = True)

Initialized
Minibatch loss at step 0: 3523.802490234375
Minibatch accuracy: 10.9
Minibatch loss at step 500: 21.427549362182617
Minibatch accuracy: 81.2
Minibatch loss at step 1000: 0.8606820702552795
Minibatch accuracy: 84.4
Minibatch loss at step 1500: 0.6477633118629456
Minibatch accuracy: 85.2
Minibatch loss at step 2000: 0.859876811504364
Minibatch accuracy: 81.2
Minibatch loss at step 2500: 0.8306270837783813
Minibatch accuracy: 79.7
Minibatch loss at step 3000: 0.8443604707717896
Minibatch accuracy: 79.7
Test accuracy: 90.3


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [86]:
# TODO add cell for 
runHiddenNetwork(batch_size=15, use_l2 = True,prob_val=0.3)

Initialized
Minibatch loss at step 0: 3345.192626953125
Minibatch accuracy: 20.0
Validation accuracy: 9.9
Minibatch loss at step 500: 25.68628692626953
Minibatch accuracy: 66.7
Validation accuracy: 9.9
Minibatch loss at step 1000: 1.0143487453460693
Minibatch accuracy: 80.0
Validation accuracy: 9.9
Minibatch loss at step 1500: 0.8083603978157043
Minibatch accuracy: 86.7
Validation accuracy: 9.7
Minibatch loss at step 2000: 2.027414083480835
Minibatch accuracy: 66.7
Validation accuracy: 9.9
Minibatch loss at step 2500: 0.9328392744064331
Minibatch accuracy: 86.7
Validation accuracy: 9.6
Minibatch loss at step 3000: 0.9668338894844055
Minibatch accuracy: 86.7
Validation accuracy: 9.7
Test accuracy: 78.5


In [17]:
runHiddenNetwork(batch_size=15, use_l2 = False)

Initialized
Minibatch loss at step 0: 432.0465393066406
Minibatch accuracy: 6.7
Validation accuracy: 9.9
Minibatch loss at step 500: 562.5569458007812
Minibatch accuracy: 46.7
Validation accuracy: 10.1
Minibatch loss at step 1000: 42.18056869506836
Minibatch accuracy: 73.3
Validation accuracy: 9.5
Minibatch loss at step 1500: 5.027500629425049
Minibatch accuracy: 80.0
Validation accuracy: 10.0
Minibatch loss at step 2000: 360.86163330078125
Minibatch accuracy: 40.0
Validation accuracy: 10.0
Minibatch loss at step 2500: 51.16738510131836
Minibatch accuracy: 66.7
Validation accuracy: 10.2
Minibatch loss at step 3000: 13.242948532104492
Minibatch accuracy: 53.3
Validation accuracy: 9.8
Test accuracy: 66.1


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [132]:
runHiddenNetwork(batch_size=128, use_l2 = True, use_dropout=True, prob_val=1)

Initialized
Minibatch loss at step 0: 3465.649169921875
Minibatch accuracy: 14.1
Validation accuracy: 9.7
Minibatch loss at step 500: 21.447690963745117
Minibatch accuracy: 82.0
Validation accuracy: 9.9
Minibatch loss at step 1000: 0.8571628928184509
Minibatch accuracy: 85.2
Validation accuracy: 9.8
Minibatch loss at step 1500: 0.6506479382514954
Minibatch accuracy: 85.2
Validation accuracy: 9.9
Minibatch loss at step 2000: 0.8610601425170898
Minibatch accuracy: 80.5
Validation accuracy: 9.8
Minibatch loss at step 2500: 0.8247969150543213
Minibatch accuracy: 79.7
Validation accuracy: 9.9
Minibatch loss at step 3000: 0.8407757878303528
Minibatch accuracy: 79.7
Validation accuracy: 9.7
Test accuracy: 90.2


In [129]:
runHiddenNetwork(batch_size=128, use_l2 = False, use_dropout=False)

Initialized
Minibatch loss at step 0: 282.3809814453125
Minibatch accuracy: 14.1
Validation accuracy: 10.2
Minibatch loss at step 500: 26.93463897705078
Minibatch accuracy: 74.2
Validation accuracy: 9.8
Minibatch loss at step 1000: 11.227489471435547
Minibatch accuracy: 81.2
Validation accuracy: 9.8
Minibatch loss at step 1500: 5.092199325561523
Minibatch accuracy: 87.5
Validation accuracy: 9.9
Minibatch loss at step 2000: 2.906925678253174
Minibatch accuracy: 80.5
Validation accuracy: 9.7
Minibatch loss at step 2500: 7.554278373718262
Minibatch accuracy: 76.6
Validation accuracy: 9.9
Minibatch loss at step 3000: 5.419881820678711
Minibatch accuracy: 76.6
Validation accuracy: 9.8
Test accuracy: 87.7


In [130]:
runHiddenNetwork(batch_size=128, use_l2 = True, use_dropout=False)

Initialized
Minibatch loss at step 0: 3495.68896484375
Minibatch accuracy: 16.4
Validation accuracy: 10.1
Minibatch loss at step 500: 21.380821228027344
Minibatch accuracy: 79.7
Validation accuracy: 9.8
Minibatch loss at step 1000: 0.8532900214195251
Minibatch accuracy: 85.9
Validation accuracy: 9.8
Minibatch loss at step 1500: 0.6555989980697632
Minibatch accuracy: 84.4
Validation accuracy: 9.9
Minibatch loss at step 2000: 0.8512094020843506
Minibatch accuracy: 80.5
Validation accuracy: 9.8
Minibatch loss at step 2500: 0.8277495503425598
Minibatch accuracy: 80.5
Validation accuracy: 9.9
Minibatch loss at step 3000: 0.8403833508491516
Minibatch accuracy: 79.7
Validation accuracy: 9.7
Test accuracy: 90.3


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [265]:
batch_size = 10

hidden_nodes_1 = 1024
hidden_nodes_2 = int(hidden_nodes_1 * 0.5)
hidden_nodes_3 = int(hidden_nodes_1 * np.power(0.5, 2))
hidden_nodes_4 = int(hidden_nodes_1 * np.power(0.5, 3))


print(num_nodes_1)
print(num_nodes_2)
print(num_nodes_3)
print(image_size * image_size)

def runHiddenNetworkMulti(batch_size=128, num_steps = 3001) :
    graphRelu = tf.Graph()
    with graphRelu.as_default():
        keep_prob = tf.placeholder(tf.float32)
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)
        # Variables.
        
        
        
        weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes_1]))
        biases_1 = tf.Variable(tf.zeros([num_nodes_1]))
        weights_2 = tf.Variable( tf.truncated_normal([num_nodes_1, num_nodes_2]))
        biases_2 = tf.Variable(tf.zeros([num_nodes_2]))
        weights_3 = tf.Variable( tf.truncated_normal([num_nodes_2, num_nodes_3]))
        biases_3 = tf.Variable(tf.zeros([num_nodes_3]))

        weights_4 = tf.Variable( tf.truncated_normal([num_nodes_3, num_labels]))
        biases_4 = tf.Variable(tf.zeros([num_labels]))
        
        logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
        relu_layer_1 = tf.nn.relu(logits_1)  
        logits_2 = tf.matmul(relu_layer_1, weights_2) + biases_2     
        relu_layer_2 = tf.nn.relu(logits_2)  
        logits_3 = tf.matmul(relu_layer_2, weights_3) + biases_3
        relu_layer_3 = tf.nn.relu(logits_3)  
        logits_4 = tf.matmul(relu_layer_3, weights_4) + biases_4
        
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits_4,labels=tf_train_labels))
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
        train_prediction = tf.nn.softmax(logits_4)
        
        
        # Predictions for test
        logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
        relu_layer_1 = tf.nn.relu(logits_1)
        logits_2 = tf.matmul(relu_layer_1, weights_2) + biases_2
        relu_layer_2 = tf.nn.relu(logits_2)
        logits_3 = tf.matmul(relu_layer_2, weights_3) + biases_3
        relu_layer_3 = tf.nn.relu(logits_3)
        logits_4 = tf.matmul(relu_layer_3, weights_4) + biases_4
        test_prediction =  tf.nn.softmax(logits_4)   

        
        with tf.Session(graph=graphRelu) as session:
            tf.initialize_all_variables().run()
            print("Initialized")
            for step in range(num_steps):
                # Pick an offset within the training data, which has been randomized.
                # Note: we could use better randomization across epochs.
                offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
                # Generate a minibatch.
                batch_data = train_dataset[offset:(offset + batch_size), :]
                batch_labels = train_labels[offset:(offset + batch_size), :]
                # Prepare a dictionary telling the session where to feed the minibatch.
                # The key of the dictionary is the placeholder node of the graph to be fed,
                # and the value is the numpy array to feed to it.
                feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:}
                _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
                if (step % 500 == 0):
                    print("Minibatch loss at step {}: {}".format(step, l))
                    print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))


1024
512
256
784


In [266]:
runHiddenNetworkMulti(batch_size=128)

Initialized
Minibatch loss at step 0: 11.053153038024902
Minibatch accuracy: 10.2
Minibatch loss at step 500: nan
Minibatch accuracy: 9.4
Minibatch loss at step 1000: nan
Minibatch accuracy: 8.6
Minibatch loss at step 1500: nan
Minibatch accuracy: 11.7
Minibatch loss at step 2000: nan
Minibatch accuracy: 6.2
Minibatch loss at step 2500: nan
Minibatch accuracy: 10.2
Minibatch loss at step 3000: nan
Minibatch accuracy: 7.8
Test accuracy: 10.0
